<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : Pandas - Resolução do Exercício 4.

# Prática adicional sobre joins com pandas

Vamos trabalhar com um conjunto de dados [Coupon Redemption_SMOTE_ Feature Selection](https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data) de campanha de marketing. É representado por um conjunto de tabelas relacionais:

* train: relaciona o id do consumidor, cupons de desconto, campanha de marketing e contém a variável de destino 'redemption_status', que informa se o cupom foi resgatado ou não.

* campaign_data: data de início e término da campanha e tipo de campanha

* customer_demographics: faixa etária, estado civil, tamanho da família, número de filhos, nível de renda, proprietário / inquilino / hipoteca

* coupon_item_mapping: cada cupom oferece benefícios em diferentes itens

* item_data: marca, tipo de marca e categoria do item

* customer_transaction_data: dados sobre compras feitas pelos consumidores. Não sabemos em qual compra utilizaram o cupom de desconto, talvez a promoção funcione para mais de uma compra.

<img src='img/Schema.png' align='center' width=40%/>

In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
train = pd.read_csv('../Data/train.csv')
cust_dem = pd.read_csv('../Data/customer_demographics.csv')
trans = pd.read_csv('../Data/customer_transaction_data.csv')
coupon_item = pd.read_csv('../Data/coupon_item_mapping.csv')
item_data = pd.read_csv('../Data/item_data.csv')
campaign_data = pd.read_csv('../Data/campaign_data.csv')

## Exercício 1 - Train dataframe

* Olhe para o dataframe e verifique se há valores nulos.

In [3]:
display(train.head(3))
print('\nHá dados nulos:',train.isnull().any().any())

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0



Há dados nulos: False


## Exercício 2 - Customer demographics dataframe

* Verifique se há nulos
* Olhe para os dtypes
* Impute valores nulos a partir do valor modal de cada coluna
* Traga as colunas de variáveis categóricas para o dtype 'category'

In [4]:
# cust_dem
display(cust_dem.head(3))
print('\n-------------------------')
print('Dtypes:\n') 
print(cust_dem.dtypes)
print('\n-------------------------')
print('Nulos:\n')      
print(cust_dem.isnull().sum())

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3



-------------------------
Dtypes:

customer_id        int64
age_range         object
marital_status    object
rented             int64
family_size       object
no_of_children    object
income_bracket     int64
dtype: object

-------------------------
Nulos:

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64


Atribuímos valores ausentes para a moda:

In [5]:
# Imputamos valores faltantes
imp = SimpleImputer(strategy='most_frequent')
cust_dem[['marital_status','no_of_children']] = imp.fit_transform(cust_dem[['marital_status','no_of_children']])
print('Nulos:\n')      
print(cust_dem.isnull().sum())

Nulos:

customer_id       0
age_range         0
marital_status    0
rented            0
family_size       0
no_of_children    0
income_bracket    0
dtype: int64


Convertemos as variáveis para o tipo categórico:

In [6]:
cust_dem.iloc[:,1:]=cust_dem.iloc[:,1:].astype('category')
print('Dtypes:\n') 
print(cust_dem.dtypes)

Dtypes:

customer_id          int64
age_range           object
marital_status      object
rented            category
family_size         object
no_of_children      object
income_bracket    category
dtype: object


## Exercício 3 - Transactions dataframe

* Verifique se há nulos
* Veja os dtypes
* Remova a coluna `dates`

In [7]:
# trans data
display(trans.head(3))
print('\nHá nulos:',trans.isnull().any().any())
print('\nDtypes:\n') 
print(trans.dtypes)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0



Há nulos: False

Dtypes:

date                object
customer_id          int64
item_id              int64
quantity             int64
selling_price      float64
other_discount     float64
coupon_discount    float64
dtype: object


In [8]:
trans = trans.drop('date', axis = 1)

In [9]:
trans.head(3)
print('\nDtypes:\n') 
print(trans.dtypes)


Dtypes:

customer_id          int64
item_id              int64
quantity             int64
selling_price      float64
other_discount     float64
coupon_discount    float64
dtype: object


## Exercício 4 - Coupon-Item dataframe
* verifique se há dados nulos

In [10]:
# Coupon item map
display(coupon_item.head(3))
print('\nHá nulos:',coupon_item.isnull().any().any())

,coupon_id,item_id
0,105,37
1,107,75
2,494,76



Há nulos: False


## Exercício 5 - Item dataframe

* Verifique se há nulos
* Converta colunas categóricas para o dtype `category`

In [11]:
# Item Data
display(item_data.head(3))
item_data[['brand','brand_type','category']]=item_data[['brand','brand_type','category']].astype('category')
print(item_data.dtypes)
print('\nHá nulos:',item_data.isnull().any().any())

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery


item_id          int64
brand         category
brand_type    category
category      category
dtype: object

Há nulos: False


## Exercício 6 - Campaign dataframe
* Verifique se há valores nulos.
* Converta as colunas `start_date` e `end_date` para `TimeStamp`.
* Calcule o período das promoções como `start_date` - `end_date`.
* Remova `start_date` e `end_date`.

In [12]:
display(campaign_data.head(3))
print(campaign_data.dtypes)
print('\nHá nulos:',campaign_data.isnull().any().any())

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13


campaign_id       int64
campaign_type    object
start_date       object
end_date         object
dtype: object

Há nulos: False


Calculamos quanto tempo a promoção durou:

In [13]:
# calculamos o período de promoção

start_date = pd.to_datetime(campaign_data['start_date'], dayfirst = True)
end_date = pd.to_datetime(campaign_data['end_date'], dayfirst = True)

campaign_data['duration'] = end_date - start_date # Período de validade do cupom
campaign_data['duration'] = campaign_data['duration'].apply(lambda x: x.days)

campaign_data['campaign_type'] = campaign_data['campaign_type'].astype('category')

campaign_data['month'] = start_date.apply(lambda x : x.month).astype('category')
campaign_data['year'] = start_date.apply(lambda x : x.year).astype('category')

/var/folders/cz/8wxdxxx55vz69s8s08w90yvh0000gn/T/ipykernel_32110/1453609944.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  start_date = pd.to_datetime(campaign_data['start_date'], dayfirst = True)
/var/folders/cz/8wxdxxx55vz69s8s08w90yvh0000gn/T/ipykernel_32110/1453609944.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  end_date = pd.to_datetime(campaign_data['end_date'], dayfirst = True)


In [14]:
campaign_data = campaign_data.drop(['start_date','end_date'], axis = 1)

display(campaign_data.head(3))
print(campaign_data.dtypes)

,campaign_id,campaign_type,duration,month,year
0,24,Y,60,10,2013
1,25,Y,32,10,2013
2,20,Y,70,9,2013


campaign_id         int64
campaign_type    category
duration            int64
month            category
year             category
dtype: object


## Exercício 7 - Merge Dataframes

1. Faça um merge entre train e campaign_data. Qual coluna fica em `on`? Quais são os critérios em `how`?

2. Combine o resultado do exercício $1$) com customer_demographics. Qual coluna fica em 'on'? Quais são os critérios em 'how'?

3. Verifique os valores nulos obtidos em $2$.

4. Imputamos os valores ausentes nas colunas demográficas. Para isso, amostramos com reposição da distribuição conjunta dos atributos demográficos observados.

5. Merge com transações médias por assunto:

* Faça um groupby por consumidor no dataframe da transação

* Adicione o valor médio às variáveis contínuas (`selling_price`, `quantity`, etc)

* merge com o DataFrame obtido em 4. Qual coluna fica em `on`? Qual critério entra em `how`?

6. Merge com transações médias por item

* a. Faça um agrupamento por item no dataframe da transação

* b. Calcule o valor médio de variáveis contínuas

* c. Faça um merge com o dataframe `coupon_item`

* d. Faça um merge de c. com item_data. Qual coluna fica em `on`? Quais são os critérios em `how`?

* e. Faça um agrupamento do dataframe d.: agrupando por `coupon_id`. Para cada grupo, calcule a moda das variáveis categóricas 'brand','brand_type,'category'.

* f. Faça um agrupamento do dataframe d. agrupando por `coupon_id`. Para cada grupo, calcule a média das variáveis contínuas.

* g. Faça uma fusão de d. com e. e então com f. Qual coluna fica em `on`? Quais são os critérios em `how`?

7. Salve o dataframe resultante em um csv para prática de classe.

In [15]:
train.shape, campaign_data.shape, cust_dem.shape, trans.shape, coupon_item.shape, item_data.shape

((78369, 5), (28, 5), (760, 7), (1324566, 6), (92663, 2), (74066, 4))

<b> Fazemos um merge entre os dados de  train e campaign data</b>

In [16]:
data = pd.merge(train, campaign_data, on = 'campaign_id', how = 'left')
print(data.shape)
data.isnull().sum()

(78369, 9)


id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
campaign_type        0
duration             0
month                0
year                 0
dtype: int64

<b> Merge com dados demográficos do consumidor</b>

In [17]:
data = pd.merge(data,cust_dem,on = 'customer_id', how = 'left')
data.shape

(78369, 15)

In [18]:
data.isnull().sum()

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
campaign_type            0
duration                 0
month                    0
year                     0
age_range            34708
marital_status       34708
rented               34708
family_size          34708
no_of_children       34708
income_bracket       34708
dtype: int64

<b> Imputamos valores ausentes</b>

Vamos obter uma amostra da distribuição conjunta de variáveis com valores ausentes. Desta forma, preservamos a variabilidade dos dados (não imputamos tudo pela moda) e a estrutura das correlações entre as variáveis (não atribuímos um valor de age_range incompatível com um de family_size).

In [19]:
# Nós imputamos valores ausentes por amostragem com reposicionamento dos dados observados
# Desta forma preservamos a estrutura de correlação observada entre as colunas demográficas.

impute_columns = ['age_range','marital_status','rented','family_size','no_of_children','income_bracket']

null_mask = data[impute_columns].isnull().all(axis = 1)
random_samples = data.loc[~null_mask,impute_columns].sample(n = null_mask.sum(), replace = True)
data.loc[null_mask,impute_columns] = random_samples.values

data.isnull().sum()

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
campaign_type        0
duration             0
month                0
year                 0
age_range            0
marital_status       0
rented               0
family_size          0
no_of_children       0
income_bracket       0
dtype: int64

<b> Merge com informações de transação média por cliente </b>

Acrescentamos as informações da atividade de compra típica feita por cada consumidor.

In [20]:
# Merge transactions: cálculo da média dos valores de transação por assunto
trans_customer = trans.groupby('customer_id')[['quantity','selling_price','other_discount','coupon_discount']].agg('mean')
trans_customer.columns = ['mean_'+ col +'_cust' for col in trans_customer.columns]
trans_customer = trans_customer.reset_index();

In [21]:
data = pd.merge(data,trans_customer, on = 'customer_id', how = 'left')

In [22]:
data.shape

(78369, 19)

In [23]:
data.head(3)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,duration,month,year,age_range,marital_status,rented,family_size,no_of_children,income_bracket,mean_quantity_cust,mean_selling_price_cust,mean_other_discount_cust,mean_coupon_discount_cust
0,1,13,27,1053,0,X,47,5,2013,46-55,Married,0,1,1,5,340.487097,184.260484,-33.168935,-0.287258
1,2,13,116,48,0,X,47,5,2013,36-45,Married,0,2,1,3,31.540260,234.247013,-27.699169,-3.215039
2,6,9,635,205,0,Y,32,3,2013,46-55,Married,0,2,1,7,1.392784,121.094495,-17.795660,-2.212082


<b> Média de transações por item </b>

Temos o mapeamento entre cupom e item. Vamos adicionar informações sobre quanto cada item é consumido.

In [24]:
# Merge with item data
trans_item=trans.groupby('item_id')[['quantity','selling_price','other_discount','coupon_discount']].mean()
trans_item.columns=['mean_' + col + '_item' for col in trans_item.columns]
trans_item=trans_item.reset_index()
trans_item.head(3)


,item_id,mean_quantity_item,mean_selling_price_item,mean_other_discount_item,mean_coupon_discount_item
0,1,1.0,124.31,0.0,0.0
1,2,1.0,35.26,0.0,0.0
2,3,1.0,56.64,0.0,0.0


In [25]:
coupon_item_trans = pd.merge(coupon_item,trans_item,on = 'item_id', how = 'inner')
coupon_item_trans_brand = pd.merge(coupon_item_trans,item_data,on = 'item_id', how = 'left')
coupon_item_trans_brand.head(3)

,coupon_id,item_id,mean_quantity_item,mean_selling_price_item,mean_other_discount_item,mean_coupon_discount_item,brand,brand_type,category
0,105,37,2.285714,113.12,-17.557143,0.0,56,Local,Grocery
1,6,37,2.285714,113.12,-17.557143,0.0,56,Local,Grocery
2,22,37,2.285714,113.12,-17.557143,0.0,56,Local,Grocery


In [26]:
coupon_item_trans_brand.dtypes

coupon_id                       int64
item_id                         int64
mean_quantity_item            float64
mean_selling_price_item       float64
mean_other_discount_item      float64
mean_coupon_discount_item     float64
brand                        category
brand_type                   category
category                     category
dtype: object

In [27]:
gb = coupon_item_trans_brand.groupby('coupon_id')
coupon_category = gb['category'].apply(lambda x:x.value_counts().index[0])
coupon_brand_type = gb['brand_type'].apply(lambda x:x.value_counts().index[0])
coupon_brand = gb['brand'].apply(lambda x:x.value_counts().index[0])

coupon_brand = pd.concat([pd.DataFrame(coupon_brand), 
                          pd.DataFrame(coupon_brand_type),
                          pd.DataFrame(coupon_category)],
                         axis = 1
                        ).reset_index()
coupon_brand.head(3)

,coupon_id,brand,brand_type,category
0,1,1475,Established,Natural Products
1,2,2084,Established,Grocery
2,3,278,Established,Grocery


In [28]:
coupon_item_sales = coupon_item_trans_brand.groupby('coupon_id')[['mean_quantity_item', 
                                                                  'mean_selling_price_item', 
                                                                  'mean_other_discount_item', 
                                                                  'mean_coupon_discount_item']
                                                                ].mean()

coupon_item_sales.columns = [col + '_coupon' for col in coupon_item_sales]
coupon_item_sales = coupon_item_sales.reset_index()
coupon_item_sales.head(3)

,coupon_id,mean_quantity_item_coupon,mean_selling_price_item_coupon,mean_other_discount_item_coupon,mean_coupon_discount_item_coupon
0,1,1.221644,100.195701,-25.273218,-1.321524
1,2,1.137500,114.354437,-23.314062,0.000000
2,3,1.121525,129.981640,-17.543797,-2.747358


In [29]:
data = pd.merge(data,coupon_item_sales, 
                on = 'coupon_id', 
                how = 'left'
               )

In [30]:
data = pd.merge(data,coupon_brand, 
                on = 'coupon_id', 
                how = 'left'
               )

In [31]:
data[['brand','brand_type','category']] = data[['brand','brand_type','category']].astype('category')
data.dtypes

id                                     int64
campaign_id                            int64
coupon_id                              int64
customer_id                            int64
redemption_status                      int64
campaign_type                       category
duration                               int64
month                               category
year                                category
age_range                             object
marital_status                        object
rented                              category
family_size                           object
no_of_children                        object
income_bracket                      category
mean_quantity_cust                   float64
mean_selling_price_cust              float64
mean_other_discount_cust             float64
mean_coupon_discount_cust            float64
mean_quantity_item_coupon            float64
mean_selling_price_item_coupon       float64
mean_other_discount_item_coupon      float64
mean_coupo

In [32]:
data.shape

(78369, 26)

In [33]:
# salvamos os dados
data.to_csv('../Data/data_preprocessed.csv')